#### LangChain Essentials Course

# LangChain Agents Intro

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil. In this example we will be using Meta's `llama3.2:1b-instruct-fp16`.

---

> ⚠️ We will be using Meta for this example allowing us to run everything via API. If you would like to use OpenAI instead, please see the [OpenAI version](https://github.com/aurelio-labs/langchain-course/blob/main/notebooks/openai/04-agents-intro-openai.ipynb) of this example.

---

## Introduction to Tools

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [23]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain_core.tools import tool


@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'. Both 'x' and 'y' are numbers."""
    return x + y


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'. Both 'x' and 'y' are numbers."""
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'. Both 'x' and 'y' are numbers."""
    return x**y


@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'. Both 'x' and 'y' are numbers."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [3]:
add

StructuredTool(name='add', description="Add 'x' and 'y'. Both 'x' and 'y' are numbers.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x0000017054C65120>)

We can see the tool name, description, and arg schema:

In [4]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'. Both 'x' and 'y' are numbers."


In [5]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'. Both 'x' and 'y' are numbers.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [6]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'. Both 'x' and 'y' are numbers.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [7]:
import json

llm_output_string = '{"x": 5, "y": 2}'  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [8]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We're going to construct a simple tool calling agent. We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You're a helpful assistant. Using the tools provided you must answer the "
                "user's questions. To use the tool you must always provide the correct JSON "
                "format for the tool input. For example, if adding two numbers you would use "
                "the `add` tool, passing the two numbers to the `x` and `y` parameters."
            ),
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

We start by initializing the 32B parameter QwQ. QwQ performs very well on reasoning, maths, and tool use. We pull the model from Ollama by switching to our terminal and executing:

```
ollama pull qwq:32b
```

Once the model has finished downloading, we initialize it in LangChain using the ChatOllama class:

In [10]:
from langchain_ollama.chat_models import ChatOllama

model_name = "qwen3:8b"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older `ConversationBufferMemory` class rather than the newer `RunnableWithMessageHistory` — the reason being that we will also be using the older `create_tool_calling_agent` and `AgentExecutor` method and class.

In the `05` chapter we will be using the newer `RunnableWithMessageHistory` class as we'll be building a custom `AgentExecutor`.

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True,  # to return Message objects
)

C:\Users\toni\AppData\Local\Temp\ipykernel_1844\4092651347.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* `llm`: as already defined
* `tools`: to be defined (just a list of our previously defined tools)
* `prompt`: as already defined
* `memory`: as already defined

In [12]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

Our `agent` by itself is like one-step of our agent execution loop. So, if we call the `agent.invoke` method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [13]:
agent.invoke(
    {
        "input": "what is 10 multiplied by 7?",
        "chat_history": memory.chat_memory.messages,
        "intermediate_steps": [],  # agent will append it's internal steps here
    }
)

[ToolAgentAction(tool='multiply', tool_input={'x': 10, 'y': 7}, log='\nInvoking: `multiply` with `{\'x\': 10, \'y\': 7}`\nresponded: <think>\nOkay, the user is asking, "what is 10 multiplied by 7?" So I need to figure out which tool to use here. Let me check the available functions. There\'s add, subtract, multiply, and exponentiate. The question is about multiplication, so the multiply function is the right choice. The parameters required are x and y, both numbers. The user provided 10 and 7. So I should call the multiply function with x=10 and y=7. Let me make sure I\'m not mixing up the order. The function\'s description says multiply x and y, so 10 multiplied by 7. Yep, that\'s correct. I\'ll format the tool call accordingly.\n</think>\n\n\n\n', message_log=[AIMessage(content='<think>\nOkay, the user is asking, "what is 10 multiplied by 7?" So I need to figure out which tool to use here. Let me check the available functions. There\'s add, subtract, multiply, and exponentiate. The q

Here, we can see the LLM has generated that we should use the `multiply` tool and the tool input should be `{"x": 10.7, "y": 7.68}`. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the `AgentExecutor` class to handle the execution loop:

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

Now let's try the same query with the executor, note that the `intermediate_steps` parameter that we added before is no longer needed as the executor handles it internally.

In [15]:
agent_executor.invoke(
    {
        "input": "what is 10 multiplied by 7?",
        "chat_history": memory.chat_memory.messages,
    }
)



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10, 'y': 7}`
responded: <think>
Okay, the user is asking, "what is 10 multiplied by 7?" So I need to figure out which tool to use here. Let me check the available functions. There's add, subtract, multiply, and exponentiate. The question is about multiplication, so the multiply function is the right choice. The parameters required are x and y, both numbers. The user provided 10 and 7. So I should call the multiply function with x=10 and y=7. Let me make sure I'm not mixing up the order. The function's description says multiply x and y, so 10 multiplied by 7. Yep, that's correct. I'll format the tool call accordingly.
</think>



70.0<think>
</think>

The result of multiplying 10 by 7 is **70**.

> Finished chain.


{'input': 'what is 10 multiplied by 7?',
 'chat_history': [HumanMessage(content='what is 10 multiplied by 7?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<think>\n</think>\n\nThe result of multiplying 10 by 7 is **70**.', additional_kwargs={}, response_metadata={})],
 'output': '<think>\n</think>\n\nThe result of multiplying 10 by 7 is **70**.'}

We can see that the `multiply` tool was invoked, producing the observation of `70.0`. After the observation was provided, we can see that the LLM then generated a final response of:

```
The result of multiplying 10 by 7 is 70.
```

This final response was generated based on the original query and the tool output (ie the _observation_). We can also confirm that this answer is accurate:

In [26]:
10 * 7

70

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [16]:
agent_executor.invoke({"input": "My name is Josh", "chat_history": memory})



> Entering new AgentExecutor chain...
<think>
Okay, the user said, "My name is Josh." Let me think about how to respond.

First, I need to acknowledge their name. Since they introduced themselves, a polite response would be to greet them by name. The previous interaction was about multiplying 10 by 7, and they provided the correct answer. Now, they're sharing their name, so I should make sure to welcome them properly.

I should check if there's any specific function I need to use here. Looking at the tools provided, the functions are add, subtract, multiply, and exponentiate. Since the user is just stating their name, there's no mathematical operation needed here. The response should be a simple greeting without any tool calls.

So, the appropriate response would be something like, "Hello, Josh! Welcome." That's friendly and acknowledges their introduction. No need to use any of the functions in this case.
</think>

Hello, Josh! Welcome. How can I assist you today?

> Finished chain.

{'input': 'My name is Josh',
 'chat_history': [HumanMessage(content='what is 10 multiplied by 7?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<think>\n</think>\n\nThe result of multiplying 10 by 7 is **70**.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<think>\nOkay, the user said, "My name is Josh." Let me think about how to respond.\n\nFirst, I need to acknowledge their name. Since they introduced themselves, a polite response would be to greet them by name. The previous interaction was about multiplying 10 by 7, and they provided the correct answer. Now, they\'re sharing their name, so I should make sure to welcome them properly.\n\nI should check if there\'s any specific function I need to use here. Looking at the tools provided, the functions are add, subtract, multiply, and exponentiate. Since the user is just stating their name, there\'s no mathema

Now let's try and get the agent to perform multiple tool calls within a single execution loop:

In [ ]:
agent_executor.invoke(
    {
        "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
        "chat_history": memory,
    }
)



> Entering new AgentExecutor chain...


Let's confirm that the answer is accurate:

In [ ]:
9 + 10 - (4 * 2) ** 3

Perfect, now let's see if the agent can still recall our name:

In [17]:
agent_executor.invoke({"input": "What is my name", "chat_history": memory})



> Entering new AgentExecutor chain...
<think>
Okay, the user asked, "What is my name?" Let me check the conversation history.

Earlier, the user said, "My name is Josh," and I responded by asking how I could assist. Now they're asking for their name. Since I have access to the previous messages, I should recall that their name is Josh. I don't need to use any tools here because the information is already in the chat history. Just need to provide the answer directly without calling a function.
</think>

Your name is Josh.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10 multiplied by 7?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<think>\n</think>\n\nThe result of multiplying 10 by 7 is **70**.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Josh', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<think>\nOkay, the user said, "My name is Josh." Let me think about how to respond.\n\nFirst, I need to acknowledge their name. Since they introduced themselves, a polite response would be to greet them by name. The previous interaction was about multiplying 10 by 7, and they provided the correct answer. Now, they\'re sharing their name, so I should make sure to welcome them properly.\n\nI should check if there\'s any specific function I need to use here. Looking at the tools provided, the functions are add, subtract, multiply, and exponentiate. Since the user is just stating their name, there\'s no mathema

The agent has successfully recalled our name. Let's move on to another agent example.

## SerpAPI Weather Agent

In this example, we'll be using the same agent and executor setup as before, but we'll be adding the [SerpAPI](https://serpapi.com/users/sign_in) service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 100 searches per month.

Here we will load the `serpapi` tool directly from the prebuilt tools that LangChain provides.

In [27]:
from langchain_community.tools import TavilySearchResults
from langchain.agents import load_tools

search = TavilySearchResults()
toolbox = [search]
# toolbox = load_tools(tool_names=['serpapi'], llm=llm)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [28]:
import requests
from datetime import datetime


@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if "loc" in data:
            latitude, longitude = data["loc"].split(",")
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"


@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can create our prompt, this time we'll skip the `chat_history` part as we don't need it. However, you can add it if preferred.

In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you're a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Now we create our full `tools` list, our `agent`, and the `agent_executor`:

In [30]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

For me I have to specify to the AI as I live in the UK and alot of places in the UK also exist in USA, which is why I explicitly state to use the country in the search as well as the town / city.

In [31]:
out = agent_executor.invoke(
    {
        "input": (
            "I have a few questions, what is the date and time right now? "
            "How is the weather where I am? Please give me degrees in Celsius"
        )
    }
)



> Entering new AgentExecutor chain...


KeyboardInterrupt: 

In [ ]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

That's the correct answer, and we even get the approximate answer in Celsius despite the tool returning the temperature in Fahrenheit.

We've finished our into to LangChain Agents, in the next chapter we will be looking at how to create custom agents and executors.

---